In [1]:
import torch
import matplotlib.pyplot as plt 
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

In [18]:
with open("./names.txt") as f:
    words = f.read().splitlines()
    print(len(words), max(len(w) for w in words), min(len(w) for w in words))

chars = sorted(list(set((''.join(words)))))  #get unique characters 
stoi = {s:i+1 for i, s in enumerate(chars)}  # map char to int
stoi["."] = 0 
itos = {i:s for s,i in stoi.items()}         # map int to char
vocab_size = len(itos)


# build the dataset
block_size = 8 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])


32033 15 2
torch.Size([182625, 8]) torch.Size([182625])
torch.Size([22655, 8]) torch.Size([22655])
torch.Size([22866, 8]) torch.Size([22866])


In [31]:
n_dim = 5
n_hidden = 200
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.emb = nn.Embedding(vocab_size,n_dim)
        self.seq = nn.Sequential(
            nn.Linear(n_dim*block_size,200),
            nn.Linear(200, vocab_size)
        
        )
    def forward(self, x):
        emb = self.emb(x).view(-1, n_dim*block_size)
        return self.seq(emb)

In [130]:
model =MLP()

In [149]:
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

In [160]:
its = 1000
for i in tqdm(range(its)):
  
  # minibatch construct
    ix = torch.randint(0, Xtr.shape[0], (10,))
    logits = model(Xtr[ix])
    loss = F.cross_entropy(logits, Ytr[ix])
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i%100==0:print(loss.item())

 71%|███████   | 710/1000 [00:00<00:00, 3573.78it/s]

2.5816311836242676
3.563222885131836
2.5201170444488525
2.757434129714966
2.0465219020843506
3.040191173553467
2.8126206398010254
2.2831687927246094


100%|██████████| 1000/1000 [00:00<00:00, 3562.52it/s]

2.586101770401001
2.455247163772583


In [161]:
for _ in range(20):
    
    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      

      logits = logits = model(torch.tensor([context])) # (1,block_size,d)
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break
    
    print(''.join(itos[i] for i in out))

gssee.
maxlai.
clarro.
arlmara.
alrrnahev.
carlismukh.
oilloeh.
kaiynen.
baler.
yorisia.
erasachi.
harajlohi.
aiavee.
asingtolhlhajhuh.
elameelr.
yantoro.
ea.
ymamae.
elmavin.
ke.
